In [2]:
import cv2
import numpy as np


# 그리기 함수
def draw_ball_location(img_line, locations):
    for i in range(len(locations)-1):

        if locations[0] is None or locations[1] is None:
            continue

        cv2.line(img_line, tuple(locations[i]), tuple(locations[i+1]), (255, 255, 255), 3)

    return img_line


roi_hist = None   #추적 객체 히스토그램 저장 변수
win_name = 'Screen Draw'
termination = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

list_ball_location = [] # 현재 그리고 있는 선들의 위치
history_ball_locations = [] # 이전에 그려졌던 선들의 위치
isDraw = True

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)   # 좌우 반전
    img_draw = frame.copy()
    
    img_draw = draw_ball_location(img_draw, list_ball_location) # 현재 그리고 있는 list_ball_location에 저장된 물체의 중심점 좌표를
    
    for ball_locations in history_ball_locations: # 이전에 그렸던 물체의 자취가 저장되어 있는 history_ball_locations에 저장되어있는
        img_draw = draw_ball_location(img_draw, ball_locations)
    
    if roi_hist is not None:   # 추적 대상 객체 히스토그램 등록됨
        # 전체 영상 hsv 컬러로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # 전체 영상 히스토그램과 roi 히스토그램 역투영
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        # 역투영 결과와 초기 추적 위치로 평균 이동 추적
        ret, (x,y,w,h) = cv2.meanShift(dst, (x,y,w,h), termination)
        
        # 새로운 위치에 사각형 표시
        cv2.rectangle(img_draw, (x,y), (x+w, y+h), (0,0,255), 2)
        
        # 선택한 객체 중앙에 원 표시
        center_x = int((x+x+w)/2)
        center_y = int((y+y+h)/2)
        cv2.circle(img_draw, (center_x, center_y), 3, (0, 255, 0), -1)
        
        # 구조화 요소 커널, 사각형 (5 x 5) 생성
        k = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        # 열림 연산 적용 (주변보다 밝은 노이즈 제거)
        img_mask = cv2.morphologyEx(dst, cv2.MORPH_OPEN, k, iterations = 1)
        
        # 컬러 영상과 역투영 영상을 통합해서 출력
        result = np.hstack((img_draw, cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)))
        
        if isDraw: # isDraw가 True라면
            list_ball_location.append((center_x, center_y)) # 물체의 중심좌표를 list_ball_location에 저장한다.
            
        else: # isDraw가 False라면
            history_ball_locations.append(list_ball_location.copy()) # history_ball_location에 지금까지 그렸던 list_ball_location을
            list_ball_location.clear() # 그리고 현재 list_ball_location을 clear한다.
        
    else:   # 추적 대상 객체 히스토그램 등록 안 됨
        cv2.putText(img_draw, "Set up a pen to use.", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
        result = img_draw
        
    cv2.imshow(win_name, result)
    
    key = cv2.waitKey(1) & 0xff
    if key == 27: #ESC
        cv2.waitKey(0)
        break
    elif key == ord(' '): # 스페이스바, ROI 설정
        x,y,w,h = cv2.selectROI(win_name, frame, False)
        if w and h:   # ROI가 제대로 설정됨
            # 초기 추적 대상 위치로 roi 설정
            roi = frame[y:y+h, x:x+w]
            # roi를 HSV 컬러로 변경
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = None
            # roi에 대한 히스토그램 계산
            roi_hist = cv2.calcHist([roi], [0], mask, [180], [0,180])
            cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
        else:     # ROI 설정 안 됨
            roi_hist = None
    elif key == ord('r'):
        list_ball_location.clear()
        history_ball_locations.clear()
    elif key == ord('s'):
        isDraw = not isDraw
            
else:
    print('no camera!')
        
    cap.release()
    cv2.destroyAllWindows()